# Specific regions: Mississippi River Basin and Driftless

The purpose of this notebook is to add precalculated data for two (or more in the future) geometries that are too big to be used on the on-the-fly analysis but are interesting for the EO Wilson foundations. 

The process is the same used in other precalculated geometries: 
* In ArcGIS Online (SpecificRegions project) we extract the species and contextual data for these geometries. In the "History" of the project it's possible to see the different steps followed to get the data.
* In the present notebook, we format the biodiversity data (it needs to be provided as an array of strings for the frontend) and add the contextual data



### Python libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy
from itertools import repeat
import functools


### Utils

**getHTfromId**

In [2]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

**format_df**

In [3]:
def format_df(path, file_name, lookups_id):
    #save_steps= np.arange(1,217486,step=5000)
    ### Loop over each set of data and add all results to one dictionary that will be a text field in the Feature Service

    df = pd.read_csv(f'{path}/{file_name}')
    col_name = [col for col in df.columns if col in ['SUM_amphibians','SUM_birds','SUM_presence','SUM_reptiles']]
    df.rename(columns={'SliceNumbe':'SliceNumber',col_name[0]:'SUM'}, inplace=True)

    ### Get species area against global species range:
    lookup = getHTfromId(lookups_id)
    df = df.merge(lookup[['SliceNumber','range_area_km2']], how='left',on = 'SliceNumber')
    df['per_global'] = round(df['SUM']/df['range_area_km2']*100,2)
    df.loc[df['per_global']> 100,'per_global'] = 100 ### make max presence 100%
    
    ### Get species area against aoi area:
    df = df.merge(regions[['MOL_ID','AREA_KM2']])
    df['per_aoi'] = round(df['SUM']/df['AREA_KM2']*100,2)
    df.loc[df['per_aoi']> 100,'per_aoi'] = 100 ### make max presence 100%
    
    return df

### Connect to ArcGIS API

In [4]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [5]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [6]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


### Datasets

In [7]:
path_in = '/Users/sofia/Documents/HE_Data/SpecificRegions/inputs'
path_out = '/Users/sofia/Documents/HE_Data/SpecificRegions/outputs'

In [13]:
regions= gpd.read_file(f'{path_in}/SpecificRegions_simplified/SpecificRegions_simplified.shp')
regions

,Shape_Leng,MOL_ID,region,AREA_KM2,Shape_Le_1,Shape_Area,InPoly_FID,SimPgnFlag,MaxSimpTol,MinSimpTol,geometry
0,1.765209e+06,1,1,6.641232e+04,1.728075e+06,1.265386e+11,1,0,10000.0,10000.0,"POLYGON ((-10263667.253 5616356.843, -10246655..."
1,1.831555e+07,2,2,3.287988e+06,1.654381e+07,5.761123e+12,2,0,10000.0,10000.0,"POLYGON ((-12222459.495 6387971.751, -12216626..."


In [18]:
regions = regions[['MOL_ID','region','AREA_KM2', 'geometry']]
regions['NAME']= np.where(regions['MOL_ID']==1, 'Driftless Area Restoration Effort', 'Mississippi River Basin')
regions

,MOL_ID,region,AREA_KM2,geometry,NAME
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin


### Format Biodiversity data

In [19]:
### Ids of lookup tables for each taxa in ArcGIS online
lookups = {'amphibians':'c221a727e12048b2a6ec8e762bc5f478',
         'birds':'bcb31fd9091446a0af3cfdaed334a8da',
         'mammals':'212a3dd4665845deb5d2adf5b597aae0',
         'reptiles':'5b606a03b3fc431e8d4b9191c88bc2b9'}

# Get data for all taxa
amphibians = format_df(path_in, 'biodiversity/amphibians.csv', lookups['amphibians'])
birds = format_df(path_in, 'biodiversity/birds.csv', lookups['birds'])
mammals = format_df(path_in, 'biodiversity/mammals.csv', lookups['mammals'])
reptiles = format_df(path_in, 'biodiversity/reptiles.csv', lookups['reptiles'])


In [20]:
# Format biodiversity data
amphibians = amphibians.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('amphibians').reset_index()
birds = birds.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('birds').reset_index()
mammals = mammals.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('mammals').reset_index()
reptiles = reptiles.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('reptiles').reset_index()

In [21]:
# Merge biodiversity data
regions = pd.merge(regions, amphibians, how='left', on = 'MOL_ID')
regions = pd.merge(regions, birds, how='left', on = 'MOL_ID')
regions = pd.merge(regions, mammals, how='left', on = 'MOL_ID')
regions = pd.merge(regions, reptiles, how='left', on = 'MOL_ID')
regions.head()

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_..."
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_..."


In [23]:
regions.loc[regions['MOL_ID']==1,'amphibians'].values[0]

'[{"SliceNumber":3.0,"per_global":1.23,"per_aoi":32.34},{"SliceNumber":170.0,"per_global":1.01,"per_aoi":24.33},{"SliceNumber":175.0,"per_global":0.12,"per_aoi":4.2},{"SliceNumber":185.0,"per_global":0.6,"per_aoi":44.57},{"SliceNumber":305.0,"per_global":1.32,"per_aoi":95.49},{"SliceNumber":2188.0,"per_global":0.96,"per_aoi":16.05},{"SliceNumber":2346.0,"per_global":1.3,"per_aoi":45.88},{"SliceNumber":2366.0,"per_global":2.44,"per_aoi":53.52},{"SliceNumber":3225.0,"per_global":0.0,"per_aoi":0.0},{"SliceNumber":3228.0,"per_global":0.49,"per_aoi":49.09},{"SliceNumber":3230.0,"per_global":2.05,"per_aoi":95.66},{"SliceNumber":3247.0,"per_global":2.43,"per_aoi":84.21},{"SliceNumber":3248.0,"per_global":1.77,"per_aoi":95.67},{"SliceNumber":3252.0,"per_global":0.01,"per_aoi":0.16},{"SliceNumber":3734.0,"per_global":1.27,"per_aoi":2.17},{"SliceNumber":3768.0,"per_global":0.54,"per_aoi":21.19},{"SliceNumber":4540.0,"per_global":0.38,"per_aoi":9.92},{"SliceNumber":5060.0,"per_global":1.16,"per_a

In [24]:
regions.to_file(f"{path_out}/regions_precalculated_biodiv.geojson",driver='GeoJSON') 
regions.to_file(f"{path_out}/regions_precalculated_biodiv.shp", driver='ESRI Shapefile')

### Add contextual data

In [25]:
### Read files (Files coming from ArcGIS project SpecificRegions and exported as csv)
elu= pd.read_csv(f'{path_in}/contextual/ELU.csv')
le= pd.read_csv(f'{path_in}/contextual/LE.csv')
pop= pd.read_csv(f'{path_in}/contextual/POP.csv')
wp= pd.read_csv(f'{path_in}/contextual/wdpa_percentage.csv')

cont_lookups={'elu':'83802a7fa3d34c1fa40844fc14683966',
              'landencroach':'b0ad58e2b2404eec811b711b604b9d55'}

In [26]:
elu = elu.rename(columns ={'OBJECTID_1':'MOL_ID'})
le = le.rename(columns ={'OBJECTID_1':'MOL_ID'})
pop = pop.rename(columns ={'OBJECTID_1':'MOL_ID'})

In [27]:
wp.head(1)

,OID_,region,COUNT,AREA,Variable,Dimensions,SliceNumber,SUM,Total,percentage_wdpa
0,1,1,7308.0,7.308000e+09,WDPA_GDAL3_1_0_20210615_FILTERED_TERR01_missin...,SliceNumber,1.0,306168.402923,60146,12.15


**Ecological Land Units**

In [28]:
## Add contextual data: ELU
regions = regions.merge(elu[['MOL_ID','MAJORITY']], how='left', on = 'MOL_ID').rename(columns={'MAJORITY':'majority_land_cover_climate_regime'})

In [30]:
regions.head()

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_...",149


In [31]:
# Retrieve elu lookup table to see to see the correspondences for that elu code
elu_lookup = getHTfromId(cont_lookups['elu'])
elu_lookup.head()

,elu_code,elu,lc_type,lf_type,cr_type,ObjectId
0,301,Sub Tropical Moist Forest on Plains,Forest,Plains,Sub Tropical Moist,1
1,201,Warm Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Warm Temperate Dry,2
2,151,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Cool Temperate Dry,3
3,302,Sub Tropical Moist Cropland on Tablelands,Cropland,Tablelands,Sub Tropical Moist,4
4,152,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Tablelands,Cool Temperate Dry,5


In [32]:
# Merge in dataset the required info from lookup table
regions = regions.merge(elu_lookup[['elu_code','lc_type','cr_type']], how='left', left_on = 'majority_land_cover_climate_regime', right_on = 'elu_code')\
    .drop(columns=['elu_code'])\
    .rename(columns={'lc_type':'land_cover_majority','cr_type':'climate_regime_majority'})

In [33]:
regions

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110,Cropland,Cool Temperate Moist
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_...",149,Grassland,Cool Temperate Dry


**Land Encroachment**

In [34]:
# Each slicenumber with a type of land encroachment
le.head(1)

,OID_,MOL_ID,COUNT,AREA,Variable,Dimensions,SliceNumber,MIN,MAX,RANGE,MEAN,STD,SUM,MEDIAN,PCT90,Total,percentage_land_encroachment
0,1,1,165.0,0.011458,Variable_new,SliceNumber,1.0,0.382408,0.747417,0.365009,0.64276,0.064031,106.055444,0.661601,0.705445,106363,0.16


In [35]:
# See all land encroachment data for each mol_id
long =le[['SliceNumber','MOL_ID','percentage_land_encroachment']].pivot(index='MOL_ID', columns='SliceNumber', values='percentage_land_encroachment').reset_index().rename(columns=
                                                                                                                              {1.0:'percent_irrigated',
                                                                                                                              2.0:'percent_rainfed',
                                                                                                                              3.0:'percent_rangeland',
                                                                                                                                 4.0:'percent_urban'})

In [36]:
long.head(1)

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,1,0.16,93.5,NaN,3.61


In [37]:
# How many encroachment values are higher than 100? This can be due to the way the area is calculated. Anyway, the website will show only the le that is 
# majoritary and the % will stop at 100%
long['sum']=long.iloc[:, 1:5].sum(axis=1)
long[long['sum']>100]

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,sum


In [38]:
## Add contextual data: Land Encroachment
regions = regions.merge(long[['MOL_ID','percent_irrigated', 'percent_rainfed', 'percent_rangeland','percent_urban']],on ='MOL_ID',how='left')

In [39]:
regions

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110,Cropland,Cool Temperate Moist,0.16,93.50,NaN,3.61
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_...",149,Grassland,Cool Temperate Dry,0.99,58.78,23.79,4.41


**Population**

In [40]:
# Population table, we need the SUM attribute
pop.head(1)

,OID_,MOL_ID,COUNT,AREA,SUM
0,1,1,106077.0,7.366458,1.737478e+06


In [41]:
## Add contextual data: POP
regions = regions.merge(pop[['MOL_ID','SUM']],on ='MOL_ID',how='left')
regions.head(1)

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,SUM
0,1,1,66412.318093,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110,Cropland,Cool Temperate Moist,0.16,93.5,NaN,3.61,1.737478e+06


In [42]:
regions= regions.rename(columns ={'SUM':'population_sum'})
regions

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110,Cropland,Cool Temperate Moist,0.16,93.50,NaN,3.61,1.737478e+06
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_...",149,Grassland,Cool Temperate Dry,0.99,58.78,23.79,4.41,9.062808e+07


In [43]:
wp.head()

,OID_,region,COUNT,AREA,Variable,Dimensions,SliceNumber,SUM,Total,percentage_wdpa
0,1,1,7308.0,7.308000e+09,WDPA_GDAL3_1_0_20210615_FILTERED_TERR01_missin...,SliceNumber,1.0,3.061684e+05,60146,12.15
1,2,2,281128.0,2.811280e+11,WDPA_GDAL3_1_0_20210615_FILTERED_TERR01_missin...,SliceNumber,1.0,1.595639e+07,2794859,10.06


In [44]:
## Add contextual data: POP
regions = regions.merge(wp[['region','percentage_wdpa']],on ='region',how='left')
regions = regions.rename(columns={'percentage_wdpa':'percentage_protected'})

In [45]:
regions

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110,Cropland,Cool Temperate Moist,0.16,93.50,NaN,3.61,1.737478e+06,12.15
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_...",149,Grassland,Cool Temperate Dry,0.99,58.78,23.79,4.41,9.062808e+07,10.06


In [46]:
regions = regions.fillna(0)
regions

,MOL_ID,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected
0,1,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110,Cropland,Cool Temperate Moist,0.16,93.50,0.00,3.61,1.737478e+06,12.15
1,2,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_...",149,Grassland,Cool Temperate Dry,0.99,58.78,23.79,4.41,9.062808e+07,10.06


In [47]:
regions = regions.drop(columns='MOL_ID')
regions

,region,AREA_KM2,geometry,NAME,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected
0,1,6.641232e+04,"POLYGON ((-10263667.253 5616356.843, -10246655...",Driftless Area Restoration Effort,"[{""SliceNumber"":3.0,""per_global"":1.23,""per_aoi...","[{""SliceNumber"":36.0,""per_global"":0.72,""per_ao...","[{""SliceNumber"":446.0,""per_global"":1.29,""per_a...","[{""SliceNumber"":1048.0,""per_global"":2.58,""per_...",110,Cropland,Cool Temperate Moist,0.16,93.50,0.00,3.61,1.737478e+06,12.15
1,2,3.287988e+06,"POLYGON ((-12222459.495 6387971.751, -12216626...",Mississippi River Basin,"[{""SliceNumber"":3.0,""per_global"":45.17,""per_ao...","[{""SliceNumber"":36.0,""per_global"":27.27,""per_a...","[{""SliceNumber"":129.0,""per_global"":1.54,""per_a...","[{""SliceNumber"":278.0,""per_global"":50.34,""per_...",149,Grassland,Cool Temperate Dry,0.99,58.78,23.79,4.41,9.062808e+07,10.06


In [48]:
regions.to_file(f"{path_out}/regions_precalculated_contextual.geojson",driver='GeoJSON')
